In [2]:
import torch
import numpy as np

import pymongo

from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

mongodb_connectionstring="<connection string>"
mongodb_dbname="test"
mongodb_collname="vector_comments1"
mongodb_indexname="vector_comments1"

client = pymongo.MongoClient(mongodb_connectionstring)
collection = client[mongodb_dbname][mongodb_collname]

In [5]:
with open(path) as f:
   for l in f:
       line = l.strip().split("\t")

path = r'/Users/dongq/Downloads/vector/naver_shopping.txt'
## 출처 
## https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt

result_array = []
count = 0
with open(path) as items:
    for l in items:
       result_doc = {}
       doc = l.strip().split("\t")
       doc_vector = model.encode(doc[1]).tolist()
       result_doc['rating'] = int(doc[0])
       result_doc['comment'] = doc[1]
       result_doc['embedding'] = doc_vector
       count = count + 1
       result_array.append(result_doc)

       if count % 1000 == 0:
          res = collection.insert_many(result_array)
          result_array = []
          print(count)

       ## takes time due to encoding overhead. For faster encoding, GPU is required
       ## for demo
       ##if count == 50:
       ##   break;

if result_array != []:
   res = collection.insert_many(result_array)

10
20
30
40
50


## vector search index build
mongodb_indexname="vector_comments"

```bash
{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": {
        "type": "knnVector",
        "dimensions": 768,
        "similarity": "cosine"
      }
    }
  }
}
```

In [22]:
mongodb_dbname="test"
mongodb_collname="vector_comments"
mongodb_indexname="vector_comments"

collection = client[mongodb_dbname][mongodb_collname]

# sample searches
query = "그따위로 장사하지 마세요"
query = "상담사 남자분이 너무 친절해요"
query = "와이프가 너무 좋아합니다."
query = "추운날씨관련된 내용"
query = "더운날씨관련된 내용"
query = "음식관련"
query = "엄지척"
query = "best"
query = "worst"
query = "돈 많이 버세요"

vector_query = model.encode(query).tolist()
pipeline = [
    {
        "$search": {
            "index": mongodb_indexname,
            "knnBeta": {
                "vector": vector_query,
                "path": "embedding",
                "k": 10
            }
        }
    },
    {
        "$limit": 10
    },
    {
        "$project": {
            "embedding": 0,
            "_id": 0,
            'search_score': {
                '$meta': 'searchScore'
            }
        }
    }
]

print("*************Vector Search Query*****************")
print(query)
print("**************************************************")

print("\n")
print("*************Vector Search Result*****************")

results = collection.aggregate(pipeline)
for result in results:
    print('rating: ' + str(result['rating']))
    print('comment: ' + result['comment'])
    print('search_score: ' + str(result['search_score']))
    print("\n")

print("**************************************************")

*************Vector Search Query*****************
그따위로 장사하지 마세요
**************************************************


*************Vector Search Result*****************
rating: 2
comment: 장사 이렇게 하지 마세요.
search_score: 0.928044319152832


rating: 2
comment: 그런식으로 장사하면 않돼요 빅사이즈 쇼핑몰 에 올리지 마세요
search_score: 0.8850153684616089


rating: 1
comment: 별로입니다 장사 우기기하지마세요
search_score: 0.8839950561523438


rating: 2
comment: 장사 그런식의 서비스마인드로 하시지마세요. 최악입니다.
search_score: 0.8627434372901917


rating: 1
comment: 정말 다신 이런거 판매하지마세요 양심이 있으먼
search_score: 0.8545899391174316


rating: 1
comment: 두번다시 이런식으로 속여팔지 마세요.
search_score: 0.8529224395751953


rating: 2
comment: 여기서 구매하지마세요...
search_score: 0.8512523770332336


rating: 1
comment: 죄송하지만 왠만하면 이런건 팔지 마세요
search_score: 0.8505539894104004


rating: 1
comment: 이런 물건 팔지 맙시다.
search_score: 0.8418306112289429


rating: 2
comment: 판매하지마세여 ㅡㅡ
search_score: 0.8404841423034668


**************************************************
